# Day 07: LLM 可以执行代码 - 自主问题解决

**探索 LLM 如何不仅能编写代码，还能自主执行、调试和优化代码**

## 课程来源
- [Advent of Agents 2025 - Day 7](https://adventofagents.com/)
- [Code Execution on Vertex AI](https://cloud.google.com/vertex-ai/docs/agent-engine/code-execution)

## 学习目标

1. 理解 Code Execution 的概念和价值
2. 掌握 `BuiltInCodeExecutor` 的使用
3. 构建能够自主执行代码的 Agent
4. 了解代码执行的安全性考虑
5. 实践数据分析和数学计算场景

---
## 1. 什么是 Code Execution？

### 1.1 概念解释

**Code Execution（代码执行）** 让 LLM 能够在安全的沙箱环境中运行代码。

传统 LLM 只能 **生成** 代码，但不能 **运行** 代码。这导致：
- 数学计算可能出错（LLM 靠“猜”而不是真正计算）
- 无法验证生成的代码是否正确
- 无法处理需要实际计算的数据分析任务

Code Execution 解决了这些问题！

### 1.2 核心优势

| 传统 LLM | 带 Code Execution 的 LLM |
|----------|-------------------------|
| 数学可能算错 | 100% 精确计算 |
| 无法验证代码 | 运行即验证 |
| 需要人工反馈 | 自动发现错误并修复 |

### 1.3 ADK 中的两种 Code Execution 方式

| 方式 | 说明 |
|------|------|
| **GenAI SDK 直接调用** | 使用 `types.Tool(code_execution=...)` |
| **ADK Agent** | 使用 `code_executor=BuiltInCodeExecutor()` |

---
## 2. 环境设置

In [17]:
# ============================================================
# 环境配置
# ============================================================

import sys
import os
from pathlib import Path

# 添加项目根目录到路径
sys.path.insert(0, str(Path.cwd().parent))

# 加载 API Key
from shared import get_api_key
os.environ["GOOGLE_API_KEY"] = get_api_key()

print("✅ 环境配置完成！")
print(f"   Python 版本: {sys.version.split()[0]}")

✅ 环境配置完成！
   Python 版本: 3.12.11


In [18]:
# ============================================================
# 验证 ADK 安装
# ============================================================

import google.adk as adk
print(f"✅ google-adk 已安装")
print(f"   版本: {adk.__version__ if hasattr(adk, '__version__') else '未知'}")

✅ google-adk 已安装
   版本: 1.20.0


---
## 3. 使用 GenAI SDK 直接体验 Code Execution

在使用 ADK 之前，让我们先用底层的 GenAI SDK 体验一下 Code Execution 的强大之处。

In [19]:
# ============================================================
# 示例 1：精确数学计算
# ============================================================
# 让 LLM 计算复杂的数学问题
# 没有 Code Execution，LLM 可能会算错
# ============================================================

from google import genai
from google.genai import types

client = genai.Client()

# 配置 Code Execution 工具
code_execution_tool = types.Tool(
    code_execution=types.ToolCodeExecution()  # 启用代码执行
)

# 一个需要精确计算的问题
question = """
计算以下问题：
1. 斜波那契数列的第 20 项是多少？
2. 从 1 加到 100 的和是多少？
3. 2 的 32 次方是多少？

请用 Python 代码计算，确保结果精确。
"""

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=question,
    config=types.GenerateContentConfig(
        tools=[code_execution_tool]
    )
)

print("🔢 数学计算结果：")
print("=" * 60)
print(response.text)

🔢 数学计算结果：
好的，我来计算一下这些问题。

首先，我们需要明确“斜波那契数列”的定义。由于它不是一个标准的数学术语，我假设它的定义与斐波那契数列相似，但每个数是前三个数的和。 也就是说，如果斜波那契数列的前三个数是 1, 1, 1，那么接下来的数将是 3, 5, 9, 17, ...。

我会用 Python 代码来计算：

结果如下：

1.  斜波那契数列的第 20 项（假设前三项为 1, 1, 1）是 46499。
2.  从 1 加到 100 的和是 5050。
3.  2 的 32 次方是 4294967296。



In [20]:
# ============================================================
# 查看代码执行的详细信息
# ============================================================
# 响应中包含 LLM 生成并执行的代码
# ============================================================

print("📊 代码执行详情：")
print("=" * 60)

if response.candidates:
    for part in response.candidates[0].content.parts:
        # 检查是否是可执行代码
        if hasattr(part, 'executable_code') and part.executable_code:
            print("\n📝 生成的代码：")
            print("-" * 40)
            print(part.executable_code.code)
        
        # 检查是否是代码执行结果
        if hasattr(part, 'code_execution_result') and part.code_execution_result:
            print("\n✅ 执行结果：")
            print("-" * 40)
            print(part.code_execution_result.output)

📊 代码执行详情：

📝 生成的代码：
----------------------------------------
def tribonacci(n):
  """计算斜波那契数列的第 n 项，前三项为 1, 1, 1。"""
  if n <= 0:
    return 0
  if n <= 3:
    return 1
  
  trib = [1, 1, 1]
  for i in range(3, n):
    trib.append(trib[i-1] + trib[i-2] + trib[i-3])
  return trib[n-1]

def sum_of_numbers(n):
  """计算从 1 加到 n 的和。"""
  return n * (n + 1) // 2

def power_of_two(n):
  """计算 2 的 n 次方。"""
  return 2 ** n

# 计算斜波那契数列的第 20 项
tribonacci_20 = tribonacci(20)
print(f"斜波那契数列的第 20 项是: {tribonacci_20}")

# 计算从 1 加到 100 的和
sum_1_to_100 = sum_of_numbers(100)
print(f"从 1 加到 100 的和是: {sum_1_to_100}")

# 计算 2 的 32 次方
power_2_32 = power_of_two(32)
print(f"2 的 32 次方是: {power_2_32}")


✅ 执行结果：
----------------------------------------
斜波那契数列的第 20 项是: 46499
从 1 加到 100 的和是: 5050
2 的 32 次方是: 4294967296



In [21]:
# ============================================================
# 示例 2：数据分析任务
# ============================================================
# Code Execution 特别适合数据分析场景
# ============================================================

data_question = """
我有以下销售数据：

| 月份 | 销售额（万元） |
|------|---------------|
| 1月  | 120           |
| 2月  | 95            |
| 3月  | 150           |
| 4月  | 180           |
| 5月  | 165           |
| 6月  | 210           |

请用 Python 计算：
1. 平均月销售额
2. 最高和最低销售月份
3. 销售额的标准差
4. 环比增长率（每月相比上月的增长百分比）
"""

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=data_question,
    config=types.GenerateContentConfig(
        tools=[code_execution_tool]
    )
)

print("📈 数据分析结果：")
print("=" * 60)
print(response.text)

📈 数据分析结果：
好的，我会用 Python 来计算这些数据。首先，我们需要将数据录入 Python，然后使用相应的函数进行计算。

结果如下：

1.  **平均月销售额:** 153.33 万元
2.  **最高销售额月份:** 6月 (210 万元), **最低销售额月份:** 2月 (95 万元)
3.  **销售额的标准差:** 37.82 万元
4.  **环比增长率:**
    *   2月: -20.83%
    *   3月: 57.89%
    *   4月: 20.00%
    *   5月: -8.33%
    *   6月: 27.27%



---
## 4. 使用 ADK 创建 Code Execution Agent

现在让我们用 ADK 创建一个具有代码执行能力的 Agent。

**重要：** 在 ADK 中，代码执行不是通过 `tools` 参数，而是通过 `code_executor` 参数启用！

In [22]:
# ============================================================
# 创建 Code Execution Agent
# ============================================================
# 注意：使用 code_executor 参数，不是 tools 参数！
# ============================================================

from google.adk.agents import Agent
from google.adk.code_executors import BuiltInCodeExecutor  # 内置代码执行器

# 创建具有代码执行能力的 Agent
code_agent = Agent(
    name="code_executor",
    model="gemini-2.0-flash",
    instruction="""你是一个有用的助手，可以执行 Python 代码。

当用户询问以下内容时，使用代码执行：
- 数学计算
- 数据分析
- 算法实现
- 代码验证

始终展示你的代码并清晰地解释结果。
用中文回复。""",
    code_executor=BuiltInCodeExecutor()  # 启用代码执行器
)

print(f"✅ Code Execution Agent 创建成功！")
print(f"   名称: {code_agent.name}")
print(f"   模型: {code_agent.model}")
print(f"   代码执行器: {type(code_agent.code_executor).__name__}")

✅ Code Execution Agent 创建成功！
   名称: code_executor
   模型: gemini-2.0-flash
   代码执行器: BuiltInCodeExecutor


In [23]:
# ============================================================
# 运行 Code Execution Agent
# ============================================================

from google.adk.runners import InMemoryRunner
from google.genai.types import Content, Part

async def run_code_agent(query: str):
    """运行 Code Agent 并返回结果"""
    runner = InMemoryRunner(
        agent=code_agent,
        app_name="day_07_demo"
    )
    
    session = await runner.session_service.create_session(
        app_name="day_07_demo",
        user_id="user_1"
    )
    
    user_message = Content(
        role="user",
        parts=[Part(text=query)]
    )
    
    print(f"👤 用户: {query}")
    print("-" * 50)
    
    async for event in runner.run_async(
        user_id="user_1",
        session_id=session.id,
        new_message=user_message
    ):
        if event.content and event.content.parts:
            for part in event.content.parts:
                # 显示文本回复
                if hasattr(part, 'text') and part.text:
                    print(f"🤖 Agent: {part.text}")
    
    print("\n" + "=" * 50)

In [24]:
# ============================================================
# 测试 1: 数学计算
# ============================================================

await run_code_agent("计算圆周率 π 的前 20 位小数")

👤 用户: 计算圆周率 π 的前 20 位小数
--------------------------------------------------
🤖 Agent: 好的，可以使用 Python 计算圆周率 π 的前 20 位小数。


🤖 Agent: 因此，圆周率 π 的前 20 位小数是 3.14159265358979323846。因为Python 的 decimal 模块计算出来的是3.14159265358979323847，四舍五入到小数点后20位即为3.14159265358979323846。




In [25]:
# ============================================================
# 测试 2: 算法实现
# ============================================================

await run_code_agent("""
请实现并运行快速排序算法，对以下数组进行排序：
[64, 34, 25, 12, 22, 11, 90, 45, 33, 21]
""")

👤 用户: 
请实现并运行快速排序算法，对以下数组进行排序：
[64, 34, 25, 12, 22, 11, 90, 45, 33, 21]

--------------------------------------------------
🤖 Agent: 好的，我可以帮你用快速排序算法对给定的数组进行排序。下面是 Python 代码实现，以及排序后的结果。


🤖 Agent: ```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quick_sort(left) + middle + quick_sort(right)

arr = [64, 34, 25, 12, 22, 11, 90, 45, 33, 21]
sorted_arr = quick_sort(arr)
print(f'{sorted_arr=}')
```



In [26]:
# ============================================================
# 测试 3: 实际问题求解
# ============================================================

await run_code_agent("""
房贷计算：
- 贷款金额：100万元
- 年利率：4.2%
- 贷款期限：30年
- 还款方式：等额本息

请计算：
1. 每月还款金额
2. 总还款金额
3. 总利息支出
""")

👤 用户: 
房贷计算：
- 贷款金额：100万元
- 年利率：4.2%
- 贷款期限：30年
- 还款方式：等额本息

请计算：
1. 每月还款金额
2. 总还款金额
3. 总利息支出

--------------------------------------------------
🤖 Agent: 好的，没问题。我将使用 Python 代码计算等额本息还款方式下的房贷月供、总还款金额和总利息支出。


🤖 Agent: 计算结果如下：

1.  每月还款金额：4890.17 元
2.  总还款金额：1760461.83 元
3.  总利息支出：760461.83 元

**注意：** 以上计算结果仅供参考，实际还款金额可能因银行的具体计算方式和政策有所差异。建议您向贷款银行咨询详细的还款计划。




---
## 5. 沙箱限制和注意事项

### 5.1 沙箱限制

| 可以做 | 不可以做 |
|--------|----------|
| 数学计算 | 文件读写 |
| 字符串处理 | 网络请求 |
| 数据结构操作 | 系统命令 |
| 使用标准库 | 安装第三方包 |

### 5.2 可用的 Python 库

- `math`, `statistics` - 数学计算
- `datetime` - 日期时间处理
- `json`, `csv` - 数据格式处理
- `re` - 正则表达式
- `collections`, `itertools` - 数据结构工具

In [27]:
# ============================================================
# 演示：质数查找
# ============================================================

await run_code_agent("""
请用 Python 计算：找出 1 到 100 之间所有的质数。
""")

👤 用户: 
请用 Python 计算：找出 1 到 100 之间所有的质数。

--------------------------------------------------
🤖 Agent: 好的，我可以帮你用 Python 找出 1 到 100 之间的所有质数。以下是代码：


🤖 Agent: ```python
def is_prime(n):
  """判断一个数是否为质数"""
  if n <= 1:
    return False
  for i in range(2, int(n**0.5) + 1):
    if n % i == 0:
      return False
  return True

primes = []
for i in range(2, 101):
  if is_prime(i):
    primes.append(i)

print(primes)
```
🤖 Agent: 

这段代码首先定义了一个名为 `is_prime` 的函数，该函数接受一个整数 `n` 作为输入，并返回一个布尔值，指示该数是否为质数。该函数首先检查 `n` 是否小于或等于 1。如果是，则该函数返回 `False`，因为 1 不是质数。否则，该函数循环遍历从 2 到 `n` 的平方根的所有整数。对于每个整数 `i`，该函数检查 `n` 是否可以被 `i` 整除。如果是，则该函数返回 `False`，因为 `n` 不是质数。否则，该函数继续循环。如果该函数循环遍历了所有整数而没有找到 `n` 的任何约数，则该函数返回 `True`，因为 `n` 是质数。

然后，该代码创建一个名为 `primes` 的空列表。然后，该代码循环遍历从 2 到 100 的所有整数。对于每个整数 `i`，该代码调用 `is_prime` 函数来检查 `i` 是否为质数。如果是，则该代码将 `i` 添加到 `primes` 列表中。

最后，该代码打印 `primes` 列表。

`tool_code` 块中的代码实现了上述逻辑，运行结果如下：


🤖 Agent: 因此，1 到 100 之间的所有质数是 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71,

In [28]:
# ============================================================
# 实际案例：投资回报计算
# ============================================================

await run_code_agent("""
投资分析：

假设你从 2020 年开始每月定投 1000 元到一个年化收益率为 8% 的基金，
到 2024 年底（5 年后）：

1. 总共投入了多少本金？
2. 投资总价值是多少？
3. 总收益是多少？
""")

👤 用户: 
投资分析：

假设你从 2020 年开始每月定投 1000 元到一个年化收益率为 8% 的基金，
到 2024 年底（5 年后）：

1. 总共投入了多少本金？
2. 投资总价值是多少？
3. 总收益是多少？

--------------------------------------------------
🤖 Agent: 好的，我可以帮你计算一下。为了更准确地计算投资总价值，我会使用复利计算公式，并考虑到每月投入带来的影响。

首先，计算总投入的本金非常简单：

1. **总投入本金：**
   每月投入 1000 元，投资 5 年，也就是 5 * 12 = 60 个月。
   所以总投入本金是 1000 * 60 = 60000 元。

接下来，计算投资总价值会稍微复杂一些，需要用到复利公式。我们可以把每月的投资看作是单独的一笔投资，然后计算它们在到期时的价值，最后加总起来。

考虑到年化收益率为 8%，那么月利率大约是 8%/12。

🤖 Agent: 计算结果如下：

1.  **总投入本金：** 60000 元
2.  **投资总价值：** 73966.70 元
3.  **总收益：** 13966.70 元

因此，经过 5 年的定投，你的总投入本金是 60000 元，投资总价值约为 73966.70 元，总收益约为 13966.70 元。




In [29]:
# ============================================================
# 示例：黄金价格计算（假设已知价格）
# ============================================================
# 注意：如果需要实时数据，应该先用 google_search 获取
# 然后再用 code_execution 计算
# ============================================================

await run_code_agent("""
假设黄金当前价格是 2650 美元/盎司，
计算如果我买入 10 盎司黄金需要多少人民币？
（汇率：1 美元 = 7.2 人民币）
""")

👤 用户: 
假设黄金当前价格是 2650 美元/盎司，
计算如果我买入 10 盎司黄金需要多少人民币？
（汇率：1 美元 = 7.2 人民币）

--------------------------------------------------
🤖 Agent: 好的，我来帮你计算一下。首先，我们需要计算 10 盎司黄金的总美元价格，然后将美元价格换算成人民币。

1.  计算 10 盎司黄金的总美元价格：
    10 盎司 \* 2650 美元/盎司 = 26500 美元

2.  将美元价格换算成人民币：
    26500 美元 \* 7.2 人民币/美元 = 190800 人民币

所以，购买 10 盎司黄金需要 190800 人民币。


🤖 Agent: 因此，购买 10 盎司黄金需要 190800 元人民币。




---
## 6. 今日总结

### 核心知识点

| 概念 | 学到了什么 |
|------|------------|
| **Code Execution** | 让 LLM 在沙箱中执行代码，获得精确结果 |
| **GenAI SDK** | 使用 `types.Tool(code_execution=...)` |
| **ADK Agent** | 使用 `code_executor=BuiltInCodeExecutor()` |
| **沙箱安全** | 代码在隔离环境运行，有执行限制 |

### 代码模板

**GenAI SDK 直接调用：**
```python
from google.genai import types

tool = types.Tool(code_execution=types.ToolCodeExecution())
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=question,
    config=types.GenerateContentConfig(tools=[tool])
)
```

**ADK Agent 配置：**
```python
from google.adk.agents import Agent
from google.adk.code_executors import BuiltInCodeExecutor

agent = Agent(
    name="code_agent",
    model="gemini-2.0-flash",
    instruction="...",
    code_executor=BuiltInCodeExecutor()
)
```

### 最佳实践

| ✅ 应该做 | ❌ 不应该做 |
|----------|----------|
| 数学计算用 Code Execution | 简单算术让 LLM 猜 |
| 提供清晰的数据格式 | 模糊的自然语言描述数据 |
| 先搜索获取实时数据，再计算 | 期望 LLM 知道实时数据 |

---
## 7. 资源链接

### 官方文档
- [Code Execution on Vertex AI Agent Engine](https://cloud.google.com/vertex-ai/docs/agent-engine/code-execution) - 官方文档
- [ADK Code Executors](https://google.github.io/adk-docs/tools/code-execution/) - ADK 代码执行文档

### 教程和示例
- [Tutorial: Get Started with Code Execution](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/agents/agent_engine/tutorial_get_started_with_code_execution.ipynb) - 官方 Notebook
- [Retail AI Location Strategy](https://github.com/google/adk-samples/tree/main/python/agents/retail-ai-location-strategy) - 使用代码执行的实际案例

---

## 下一步

Day 8 将学习 **Context Management** - 上下文管理，包括 Session、Memory 和 Artifacts。继续加油！